# ポートフォリオ分析

このノートブックでは、複数銘柄のポートフォリオを構築し、分析する方法を学びます。

## 目次
1. 環境設定
2. ポートフォリオの構築
3. パフォーマンス分析
4. リスク分析
5. 業種別の分散

## 1. 環境設定

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta

# 認証情報の設定
os.environ['JQUANTS_MAIL_ADDRESS'] = 'your_email@example.com'
os.environ['JQUANTS_PASSWORD'] = 'your_password'

import pyjquants as pjq
print('準備完了')

## 2. ポートフォリオの構築

### 投資対象銘柄の選定

異なる業種から代表的な銘柄を選んで分散投資します。

In [ ]:
# 投資対象銘柄（業種分散）
portfolio_stocks = {
    '7203': {'name': 'トヨタ', 'sector': '自動車', 'shares': 100},
    '6758': {'name': 'ソニー', 'sector': '電機', 'shares': 50},
    '9984': {'name': 'ソフトバンクG', 'sector': '情報通信', 'shares': 100},
    '8306': {'name': '三菱UFJ', 'sector': '銀行', 'shares': 500},
    '4502': {'name': '武田薬品', 'sector': '医薬品', 'shares': 100},
}

# 各銘柄の情報を取得
stocks = {}
for code in portfolio_stocks:
    stocks[code] = pjq.Stock(code)

print('ポートフォリオ構成銘柄:')
print('-' * 50)
for code, info in portfolio_stocks.items():
    stock = stocks[code]
    latest = stock.latest_price
    value = float(latest.close) * info['shares']
    print(f"{info['name']:12} ({code}): {info['shares']:>4}株 × {latest.close:>8,.0f}円 = {value:>12,.0f}円")

### トレーダーでポートフォリオを構築

In [ ]:
# 5,000万円の資金でトレーダーを作成
trader = pjq.Trader(initial_cash=50_000_000)

# 各銘柄を購入
for code, info in portfolio_stocks.items():
    stock = stocks[code]
    order = trader.buy(stock, info['shares'])

# 最新日で約定
simulation_date = stocks['7203'].latest_price.date
executions = trader.simulate_fills(simulation_date)

print(f'約定件数: {len(executions)}件')
print(f'残りの現金: {trader.cash:,.0f}円')
print(f'ポートフォリオ評価額: {trader.portfolio.total_value:,.0f}円')

### ポートフォリオの構成比率

In [ ]:
# ウェイトを計算
weights = trader.portfolio.weights()

# 現金のウェイトも追加
total = trader.portfolio.total_value
cash_weight = float(trader.cash / total)

# 円グラフで可視化
fig, ax = plt.subplots(figsize=(10, 8))

labels = [portfolio_stocks[code]['name'] for code in weights.keys()] + ['Cash']
sizes = list(weights.values()) + [cash_weight]
colors = plt.cm.Set3(np.linspace(0, 1, len(labels)))

wedges, texts, autotexts = ax.pie(
    sizes, 
    labels=labels, 
    autopct='%1.1f%%',
    colors=colors,
    startangle=90
)

ax.set_title('Portfolio Composition', fontsize=14)
plt.tight_layout()
plt.show()

## 3. パフォーマンス分析

### 各銘柄の株価推移

In [ ]:
# 各銘柄の株価を取得
price_data = {}
for code, stock in stocks.items():
    prices = stock.prices[['date', 'close']].copy()
    prices.columns = ['date', portfolio_stocks[code]['name']]
    price_data[code] = prices

# 日付をキーに結合
combined = price_data['7203'].copy()
for code in list(price_data.keys())[1:]:
    combined = combined.merge(price_data[code], on='date', how='outer')

combined = combined.sort_values('date').reset_index(drop=True)
print(f'データ期間: {combined["date"].min()} ~ {combined["date"].max()}')
combined.tail()

### 正規化した株価比較

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))

stock_cols = [col for col in combined.columns if col != 'date']
for col in stock_cols:
    first_valid = combined[col].dropna().iloc[0]
    normalized = (combined[col] / first_valid) * 100
    ax.plot(combined['date'], normalized, label=col, linewidth=2)

ax.set_title('Portfolio Stocks - Normalized Performance (Base=100)', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Normalized Price')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.axhline(y=100, color='gray', linestyle='--', alpha=0.5)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. リスク分析

### 日次リターンの計算

In [ ]:
# 日次リターンを計算
returns = combined.copy()
for col in stock_cols:
    returns[col] = returns[col].pct_change() * 100

returns = returns.dropna()

# 統計量を計算
stats = pd.DataFrame({
    '平均リターン(%)': returns[stock_cols].mean(),
    '標準偏差(%)': returns[stock_cols].std(),
    '最大(%)': returns[stock_cols].max(),
    '最小(%)': returns[stock_cols].min(),
})

print('日次リターン統計:')
stats.round(3)

### 相関行列

In [ ]:
# 相関行列を計算
corr_matrix = returns[stock_cols].corr()

# ヒートマップで可視化
fig, ax = plt.subplots(figsize=(10, 8))

im = ax.imshow(corr_matrix, cmap='RdYlBu_r', vmin=-1, vmax=1)

# ラベル設定
ax.set_xticks(range(len(stock_cols)))
ax.set_yticks(range(len(stock_cols)))
ax.set_xticklabels(stock_cols, rotation=45, ha='right')
ax.set_yticklabels(stock_cols)

# 相関係数を表示
for i in range(len(stock_cols)):
    for j in range(len(stock_cols)):
        text = ax.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                       ha='center', va='center', color='black', fontsize=10)

ax.set_title('Return Correlation Matrix', fontsize=14)
plt.colorbar(im, ax=ax, shrink=0.8)
plt.tight_layout()
plt.show()

## 5. 業種別の分散

### 業種別の評価額

In [ ]:
# 業種別に集計
sector_values = {}
for code, info in portfolio_stocks.items():
    sector = info['sector']
    position = trader.position(stocks[code])
    if position:
        value = float(position.market_value)
        sector_values[sector] = sector_values.get(sector, 0) + value

print('業種別評価額:')
print('-' * 40)
for sector, value in sorted(sector_values.items(), key=lambda x: -x[1]):
    pct = value / trader.portfolio.total_value * 100
    print(f'{sector:12}: {value:>15,.0f}円 ({pct:5.1f}%)')

In [ ]:
# 業種別の円グラフ
fig, ax = plt.subplots(figsize=(10, 8))

sectors = list(sector_values.keys())
values = list(sector_values.values())
colors = plt.cm.Pastel1(np.linspace(0, 1, len(sectors)))

wedges, texts, autotexts = ax.pie(
    values,
    labels=sectors,
    autopct='%1.1f%%',
    colors=colors,
    startangle=90
)

ax.set_title('Sector Allocation', fontsize=14)
plt.tight_layout()
plt.show()

## ポートフォリオサマリー

In [ ]:
print('=' * 60)
print('ポートフォリオサマリー')
print('=' * 60)
print(f'初期投資額:       50,000,000円')
print(f'現在の評価額:     {trader.portfolio.total_value:>12,.0f}円')
print(f'  うち現金:       {trader.cash:>12,.0f}円')
print(f'  うち株式:       {trader.portfolio.total_value - trader.cash:>12,.0f}円')
print(f'含み損益:         {trader.portfolio.unrealized_pnl:>12,.0f}円')
print(f'保有銘柄数:       {len(trader.positions()):>12}銘柄')
print('=' * 60)

## まとめ

このノートブックでは以下を学びました：

- **ポートフォリオ構築**: 複数銘柄への分散投資
- **パフォーマンス分析**: 株価推移の比較
- **リスク分析**: 日次リターン、標準偏差、相関行列
- **業種分散**: セクター別のアロケーション

### 次のステップ

- より長期間のバックテスト
- リバランス戦略の検討
- ベンチマーク（TOPIX等）との比較

---

**免責事項**: このノートブックは教育目的であり、投資助言ではありません。投資判断はご自身の責任で行ってください。